<a href="https://colab.research.google.com/github/ritaja2003/Predictive-Analysis/blob/main/Predictive_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install scikit-learn pandas matplotlib seaborn tensorflow keras


In [ ]:
from google.colab import files
files.upload()


Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"ritajachatterjee2003","key":"b725e3b5c916cf6c751628dbedac07f4"}'}

In [ ]:
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json


In [ ]:
!kaggle datasets download -d shivamb/machine-predictive-maintenance-classification
!unzip machine-predictive-maintenance-classification.zip


Dataset URL: https://www.kaggle.com/datasets/shivamb/machine-predictive-maintenance-classification
License(s): CC0-1.0
  0% 0.00/137k [00:00<?, ?B/s]
100% 137k/137k [00:00<00:00, 71.9MB/s]
Archive:  machine-predictive-maintenance-classification.zip
  inflating: predictive_maintenance.csv  


In [ ]:
# Load & Explore the Data

import pandas as pd

# Load the dataset
df = pd.read_csv("predictive_maintenance.csv")

# Display the first few rows
df.head()

,UDI,Product ID,Type,Air temperature [K],Process temperature [K],Rotational speed [rpm],Torque [Nm],Tool wear [min],Target,Failure Type
0,1,M14860,M,298.1,308.6,1551,42.8,0,0,No Failure
1,2,L47181,L,298.2,308.7,1408,46.3,3,0,No Failure
2,3,L47182,L,298.1,308.5,1498,49.4,5,0,No Failure
3,4,L47183,L,298.2,308.6,1433,39.5,7,0,No Failure
4,5,L47184,L,298.2,308.7,1408,40.0,9,0,No Failure


In [ ]:
import os
print(os.getcwd())


/content


In [ ]:
"""Dataset Overview
The dataset contains:

Type: Type of equipment
Air temperature, Process temperature: Sensor data
Rotational speed, Torque, Tool wear: Operational conditions
Failure Type: Label (whether failure occurred & its type)"""

print(df.isnull().sum())

df.fillna(df.select_dtypes(include=['number']).mean(), inplace=True)



UDI                        0
Product ID                 0
Type                       0
Air temperature [K]        0
Process temperature [K]    0
Rotational speed [rpm]     0
Torque [Nm]                0
Tool wear [min]            0
Target                     0
Failure Type               0
dtype: int64


In [ ]:
if 'Type' in df.columns:
    df = pd.get_dummies(df, columns=['Type'], drop_first=True)
else:
    print("Column 'Type' not found in dataset.")




In [ ]:
if 'Failure Type' in df.columns:
    df['Failure'] = df['Failure Type'].apply(lambda x: 1 if x != 'No Failure' else 0)
    df.drop(columns=['Failure Type'], inplace=True)
else:
    print("Column 'Failure Type' not found in dataset.")



In [ ]:
print(df.dtypes)
df = pd.get_dummies(df, drop_first=True)
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()

# Select only numeric columns for scaling
numeric_columns = df.select_dtypes(include=['number']).columns
df_scaled = pd.DataFrame(scaler.fit_transform(df[numeric_columns]), columns=numeric_columns)

# If you need to keep non-numeric columns, merge them back after scaling
df_scaled['Failure'] = df['Failure']





UDI                          int64
Product ID                  object
Air temperature [K]        float64
Process temperature [K]    float64
Rotational speed [rpm]       int64
Torque [Nm]                float64
Tool wear [min]              int64
Target                       int64
Type_L                        bool
Type_M                        bool
Failure                      int64
dtype: object


In [ ]:
from sklearn.model_selection import train_test_split

X = df_scaled.drop(columns=['Failure'])
y = df_scaled['Failure']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:
#Train a random forest classifier

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

# Train Random Forest model
clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train, y_train)

# Predictions
y_pred = clf.predict(X_test)

# Evaluate Model
print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))


Accuracy: 0.996
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      1935
           1       0.97      0.91      0.94        65

    accuracy                           1.00      2000
   macro avg       0.98      0.95      0.97      2000
weighted avg       1.00      1.00      1.00      2000



In [ ]:
#Train a Deep Learning Model (LSTM)

import numpy as np

# Reshape data to fit LSTM input (samples, time steps, features)
X_train_reshaped = np.reshape(X_train.values, (X_train.shape[0], X_train.shape[1], 1))
X_test_reshaped = np.reshape(X_test.values, (X_test.shape[0], X_test.shape[1], 1))


In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

# Define LSTM model
model = Sequential([
    LSTM(50, return_sequences=True, input_shape=(X_train.shape[1], 1)),
    Dropout(0.2),
    LSTM(50),
    Dense(1, activation='sigmoid')  # Binary Classification
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train model
model.fit(X_train_reshaped, y_train, epochs=10, batch_size=32, validation_data=(X_test_reshaped, y_test))


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 7s 12ms/step - accuracy: 0.9647 - loss: 0.2766 - val_accuracy: 0.9675 - val_loss: 0.1385
Epoch 2/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - accuracy: 0.9660 - loss: 0.1341 - val_accuracy: 0.9675 - val_loss: 0.1011
Epoch 3/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - accuracy: 0.9651 - loss: 0.0852 - val_accuracy: 0.9755 - val_loss: 0.0487
Epoch 4/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 3s 9ms/step - accuracy: 0.9853 - loss: 0.0384 - val_accuracy: 0.9960 - val_loss: 0.0303
Epoch 5/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 4s 15ms/step - accuracy: 0.9967 - loss: 0.0239 - val_accuracy: 0.9960 - val_loss: 0.0268
Epoch 6/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - accuracy: 0.9975 - loss: 0.0225 - val_accuracy: 0.9960 - val_loss: 0.0275
Epoch 7/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - accuracy: 0.9980 - loss: 0.0123 - val_accuracy: 0.9960 - val_loss: 0.0264
Epoch 8/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - accuracy: 0.9978 - loss: 0.0165 - val_accuracy: 

In [ ]:
model.save("predictive_maintenance_model.h5")
from google.colab import files
files.download("predictive_maintenance_model.h5")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
#!pip install streamlit pyngrok joblib numpy pandas scikit-learn



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 54.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 73.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 5.8 MB/s eta 0:00:00


In [ ]:
##import joblib

# Assume `model` is your trained model
joblib.dump(model, "ml_model.pkl")


['ml_model.pkl']

In [ ]:
"""%%writefile app.py

import streamlit as st
import joblib
import numpy as np

# Load the trained model
model = joblib.load("ml_model.pkl")  # Make sure this file exists

# Streamlit app title
st.title("Machine Learning Model Prediction")

# Input fields for features
num_features = 5  # Change this to match your model's input
features = []

for i in range(num_features):
    feature = st.number_input(f"Feature {i + 1}", key=f"feature_{i}")
    features.append(feature)

# Predict button
if st.button("Predict"):
    input_data = np.array(features).reshape(1, -1)
    prediction = model.predict(input_data)
    st.write(f"Prediction: {prediction[0]}")"""


Writing app.py


In [ ]:
"""from pyngrok import ngrok

# Start Streamlit in the background
!streamlit run app.py &

# Expose the app via ngrok
public_url = ngrok.connect(port="8501")
print(f"Streamlit App URL: {public_url}")"""






  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.45.85.230:8501

  Stopping...


PyngrokNgrokHTTPError: ngrok client exception, API returned 400: {"error_code":102,"status_code":400,"msg":"invalid tunnel configuration","details":{"err":"yaml: unmarshal errors:\n  line 1: field port not found in type config.HTTPv2Tunnel"}}


In [ ]:
#!streamlit run app.py &>/dev/null&

In [ ]:

"""!pip install --upgrade pyngrok

from pyngrok import ngrok"""

In [ ]:
#!ngrok config add-authtoken 2tDljjaBlsR0W8iNpX1qVCdPkCk_64MWdgDnAjrZHViXPMGCJ


Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
#!streamlit run app.py --server.port 3000



2025-02-20 04:28:17.629 Port 3000 is reserved for internal development. It is strongly recommended to select an alternative port for `server.port`.

  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:3000
  Network URL: http://172.28.0.12:3000
  External URL: http://34.45.85.230:3000

  Stopping...
^C


In [ ]:
"""from pyngrok import ngrok

# Start Ngrok tunnel with the correct parameter
public_url = ngrok.connect(addr="3000")  # Change 'http' to 'addr'

print("Streamlit App URL:", public_url)"""


Streamlit App URL: NgrokTunnel: "https://9c79-34-45-85-230.ngrok-free.app" -> "http://localhost:3000"


In [ ]:

pip install flask pyngrok


In [ ]:
"""from flask import Flask, jsonify
from pyngrok import ngrok

app = Flask(__name__)

@app.route("/")
def home():
    return jsonify({"message": "Flask API is working!"})"""

"""# Start Ngrok
ngrok.set_auth_token("2tDljjaBlsR0W8iNpX1qVCdPkCk_64MWdgDnAjrZHViXPMGCJ")  # Replace with your Ngrok token
public_url = ngrok.connect(5000)  # Flask runs on port 5000
print("Public URL:", public_url)

if __name__ == "__main__":
    app.run(port=5000)"""


Public URL: NgrokTunnel: "https://2318-34-45-85-230.ngrok-free.app" -> "http://localhost:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


In [ ]:
"""python flask_app.py"""


SyntaxError: invalid syntax (<ipython-input-43-b110b3796828>, line 1)

In [ ]:
import joblib
import numpy as np
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

# Load dataset
X, y = load_iris(return_X_y=True)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train model
model = RandomForestClassifier(n_estimators=100)
model.fit(X_train, y_train)

# Save model
joblib.dump(model, "model.pkl")
print("Model saved!")


Model saved!


In [ ]:
!echo "print('Hello from train.py!')" > train.py


In [ ]:
!python train.py


Hello from train.py!


In [ ]:
import sys
!{sys.executable} -m pip install dvc



In [ ]:
!rm -rf .dvc dvc.lock dvc.yaml .gitignore
!dvc init --force


Initialized DVC repository.

You can now commit the changes to git.

+---------------------------------------------------------------------+
|                                                                     |
|        DVC has enabled anonymous aggregate usage analytics.         |
|     Read the analytics documentation (and how to opt-out) here:     |
|             <https://dvc.org/doc/user-guide/analytics>              |
|                                                                     |
+---------------------------------------------------------------------+

What's next?
------------
- Check out the documentation: <https://dvc.org/doc>
- Get help and share ideas: <https://dvc.org/chat>
- Star us on GitHub: <https://github.com/iterative/dvc>


In [ ]:
!git config --global user.email "ritaja2003@gmail.com"
!git config --global user.name "Ritaja"


In [ ]:
!dvc add /content/predictive_maintenance.csv


⠋ Checking graph
Adding...:   0% 0/1 [00:00<?, ?file/s{'info': ''}]
!
          |0.00 [00:00,     ?file/s]
                                    
!
  0% |          |0/? [00:00<?,    ?files/s]
                                           
Checking out /content/predictive_maintenance.csv:   0% 0/1 [00:00<?, ?files/s]
  0% 0/1 [00:00<?, ?files/s{'info': ''}]                                      
Adding...: 100% 1/1 [00:00<00:00, 24.43file/s{'info': ''}]

To track the changes with git, run:

	git add predictive_maintenance.csv.dvc

To enable auto staging, run:

	dvc config core.autostage true


In [ ]:
!git add predictive_maintenance.csv.dvc .gitignore

In [ ]:
!dvc remote add -d myremote /content/dvc_storage


Setting 'myremote' as a default remote.


In [ ]:
!dvc push


Pushing
!
  0% |          |0/? [00:00<?,    ?files/s]
                                           
!
  0% |          |0/? [00:00<?,    ?files/s]
                                           
Pushing to local:   0% 0/1 [00:00<?, ?file/s]
Pushing to local:   0% 0/1 [00:00<?, ?file/s{'info': ''}]
Pushing
1 file pushed


In [ ]:
!git add .
!git commit -m "Tracked dataset with DVC and pushed to remote"
!git push origin master


[master 40c44b0] Tracked dataset with DVC and pushed to remote
 30 files changed, 61053 insertions(+)
 create mode 100644 .config/.last_opt_in_prompt.yaml
 create mode 100644 .config/.last_survey_prompt.yaml
 create mode 100644 .config/.last_update_check.json
 create mode 100644 .config/active_config
 rename dvc.lock => .config/config_sentinel (100%)
 create mode 100644 .config/configurations/config_default
 create mode 100644 .config/default_configs.db
 create mode 100644 .config/gce
 create mode 100644 .config/hidden_gcloud_config_universe_descriptor_data_cache_configs.db
 create mode 100644 .config/logs/2025.02.18/14.18.50.396027.log
 create mode 100644 .config/logs/2025.02.18/14.19.21.783246.log
 create mode 100644 .config/logs/2025.02.18/14.19.31.209860.log
 create mode 100644 .config/logs/2025.02.18/14.19.35.876527.log
 create mode 100644 .config/logs/2025.02.18/14.19.45.014979.log
 create mode 100644 .config/logs/2025.02.18/14.19.45.736719.log
 create mode 100644 app.py
 delete 

In [ ]:
!dvc status


Data and pipelines are up to date.


In [ ]:
import joblib  # or pickle
model = ...  # Your trained ML model
joblib.dump(model, "model.pkl")  # Save it


['model.pkl']

In [ ]:
import joblib

def predict(data):
    model = joblib.load("model.pkl")  # Load model
    return model.predict([data])  # Modify based on input


In [ ]:
!pip install fastapi uvicorn


In [ ]:
!pip install fastapi uvicorn nest-asyncio pyngrok


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.5/71.5 kB 3.1 MB/s eta 0:00:00


In [ ]:
from fastapi import FastAPI
import joblib
from pydantic import BaseModel
import numpy as np
import uvicorn
import nest_asyncio
from pyngrok import ngrok

# Load model (ensure you have 'model.pkl' in your Colab session)
model = joblib.load("model.pkl")

# Initialize FastAPI
app = FastAPI()

# Define input data format
class ModelInput(BaseModel):
    features: list

@app.post("/predict")
def predict(data: ModelInput):
    features = np.array(data.features).reshape(1, -1)
    prediction = model.predict(features).tolist()
    return {"prediction": prediction}

# **Get your authtoken from https://dashboard.ngrok.com/auth**
# **Set your ngrok authtoken**
ngrok.set_auth_token("2tDljjaBlsR0W8iNpX1qVCdPkCk_64MWdgDnAjrZHViXPMGCJ")  # Replace with your actual authtoken

# Run Uvicorn in the background
ngrok_tunnel = ngrok.connect(8000)
print(f"Public URL: {ngrok_tunnel.public_url}")

# Apply nest_asyncio to avoid runtime issues in Colab
nest_asyncio.apply()

uvicorn.run(app, host="0.0.0.0", port=8000)


INFO:     Started server process [187]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


Public URL: https://b584-34-125-154-18.ngrok-free.app


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [187]


In [ ]:
import requests

url = "http://xyz.ngrok.io/predict"  # Replace with the ngrok URL
data = {"features": [5.1, 3.5, 1.4, 0.2]}
response = requests.post(url, json=data)

print(response.json())  # Example output: {'prediction': [0]}


ERROR:asyncio:Task exception was never retrieved
future: <Task finished name='Task-1' coro=<Server.serve() done, defined at /usr/local/lib/python3.11/dist-packages/uvicorn/server.py:68> exception=KeyboardInterrupt()>
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/uvicorn/main.py", line 579, in run
    server.run()
  File "/usr/local/lib/python3.11/dist-packages/uvicorn/server.py", line 66, in run
    return asyncio.run(self.serve(sockets=sockets))
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/nest_asyncio.py", line 30, in run
    return loop.run_until_complete(task)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/nest_asyncio.py", line 92, in run_until_complete
    self._run_once()
  File "/usr/local/lib/python3.11/dist-packages/nest_asyncio.py", line 133, in _run_once
    handle._run()
  File "/usr/lib/python3.11/asyncio/events.py", line 84, in _run
    se

JSONDecodeError: Expecting value: line 1 column 1 (char 0)